In [1]:
# Import dependencies
import pandas as pd
import gmaps
import requests
# Import the API key
from config import g_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Zhangye,CN,2022-02-28 21:53:51,38.9342,100.4517,26.96,29,11,5.75
1,1,Venado Tuerto,AR,2022-02-28 22:00:24,-33.7456,-61.9688,74.10,38,0,7.54
2,2,Catamarca,AR,2022-02-28 22:00:26,-28.4696,-65.7852,67.19,98,100,8.03
3,3,Bonthe,SL,2022-02-28 22:00:27,7.5264,-12.5050,76.37,80,33,3.58
4,4,Tuktoyaktuk,CA,2022-02-28 21:57:40,69.4541,-133.0374,-4.95,84,4,12.30


In [3]:
# Configure gmaps to use your Google API key
gmaps.configure(api_key = g_key)

In [4]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure()
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp], dissipating=False, max_intensity=300, point_radius=4)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [5]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data
fig

Figure(layout=FigureLayout(height='420px'))

In [6]:
# Heatmap of percent cloudiness
locations = city_data_df[["Lat", "Lng"]]
clouds = city_data_df["Cloudiness"]
fig = gmaps.figure(center=(30.0,31.0), zoom_level=1.5)
heat_layer=gmaps.heatmap_layer(locations, weights=clouds, dissipating=False, max_intensity=300, point_radius=4)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
# Heatmap of windspeed
locations = city_data_df[["Lat", "Lng"]]
wind = city_data_df["Wind Speed"]
fig = gmaps.figure(center=(30.0,31.0), zoom_level=1.5)
heat_layer=gmaps.heatmap_layer(locations, weights=wind, dissipating=False, max_intensity=300, point_radius=4)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
# Ask the customer to add a minimum and maximum temperature value
min_temp = float(input("What is the minimum temperature you would like for your trip?"))
max_temps = float(input("What is the maximum temperature you would like for your trip?"))

In [9]:
# Filter the dataset to find the cities that fit the criteria
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temps) & (city_data_df["Max Temp"] >= min_temp)]

preferred_cities_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
1,1,Venado Tuerto,AR,2022-02-28 22:00:24,-33.7456,-61.9688,74.10,38,0,7.54
3,3,Bonthe,SL,2022-02-28 22:00:27,7.5264,-12.5050,76.37,80,33,3.58
5,5,Labuhan,ID,2022-02-28 22:00:29,-6.8844,112.2051,77.00,89,91,5.21
13,13,Salinopolis,BR,2022-02-28 21:57:58,-0.6136,-47.3561,79.77,82,94,9.24
16,16,Puerto Ayora,EC,2022-02-28 22:00:38,-0.7393,-90.3518,80.55,87,83,5.01


In [10]:
preferred_cities_df.count()

City_ID       217
City          217
Country       216
Date          217
Lat           217
Lng           217
Max Temp      217
Humidity      217
Cloudiness    217
Wind Speed    217
dtype: int64

In [11]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)


,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Venado Tuerto,AR,74.10,-33.7456,-61.9688,
3,Bonthe,SL,76.37,7.5264,-12.5050,
5,Labuhan,ID,77.00,-6.8844,112.2051,
13,Salinopolis,BR,79.77,-0.6136,-47.3561,
16,Puerto Ayora,EC,80.55,-0.7393,-90.3518,
22,Bengkulu,ID,75.97,-3.8004,102.2655,
26,Boa Vista,BR,86.79,2.8197,-60.6733,
30,Atuona,PF,78.84,-9.8000,-139.0333,
31,Butaritari,KI,81.99,3.0707,172.7902,
33,Saint-Philippe,RE,78.42,-21.3585,55.7679,


In [12]:
# Set parameters to search for a hotel
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [14]:
# Iterate throught the DataFrame
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary
    params["location"] = f"{lat}, {lng}"

    # Use the search term: "lodging" and out latitude and longitude
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from your search
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name
    try: 
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except(IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [15]:
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Venado Tuerto,AR,74.10,-33.7456,-61.9688,La Bianca.
3,Bonthe,SL,76.37,7.5264,-12.5050,Bonthe Holiday Village
5,Labuhan,ID,77.00,-6.8844,112.2051,DoubleFAG
13,Salinopolis,BR,79.77,-0.6136,-47.3561,Salinas Park Resort Maçarico AP101
16,Puerto Ayora,EC,80.55,-0.7393,-90.3518,Finch Bay Galapagos Hotel
22,Bengkulu,ID,75.97,-3.8004,102.2655,Grage Hotel Bengkulu
26,Boa Vista,BR,86.79,2.8197,-60.6733,Aipana Plaza Hotel
30,Atuona,PF,78.84,-9.8000,-139.0333,Villa Enata
31,Butaritari,KI,81.99,3.0707,172.7902,Isles Sunset Lodge
33,Saint-Philippe,RE,78.42,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""


In [17]:
# Add a heatmap of temperature for the vacation spots
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False, max_intensity=300, point_radius=4)

marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data
fig

Figure(layout=FigureLayout(height='420px'))

In [18]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

In [19]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [20]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))